In [233]:
import yfinance as yf
import datetime
import pandas as pd
import numpy as np

In [234]:
start_date = datetime.datetime(2015, 1, 1)
end_date = datetime.datetime(2019, 12, 31)
# NVDA (Nvidia), AAPL (Apple), SPY (S&P 500 index)
ticker = "NVDA"
NEUTRAL_BENCHMARK = 0.005

In [235]:
train_data = yf.download(ticker, start=start_date, end=end_date)
test_data = yf.download(ticker, start=end_date, end=datetime.datetime(2022, 10, 20))

[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed


In [236]:
train_data["daily_return"] = train_data["Adj Close"].pct_change()
train_data["state"] = np.select(
    [train_data["daily_return"] > NEUTRAL_BENCHMARK, train_data["daily_return"] < -1 * NEUTRAL_BENCHMARK],
    ["up", "down"],
    default="neutral"
)
train_data.head(15)

,Open,High,Low,Close,Adj Close,Volume,daily_return,state
Date,,,,,,,,
2015-01-02,5.0325,5.0700,4.9525,5.0325,4.833229,11368000,NaN,neutral
2015-01-05,5.0325,5.0475,4.9250,4.9475,4.751596,19795200,-0.016890,down
2015-01-06,4.9550,4.9600,4.7925,4.7975,4.607536,19776400,-0.030318,down
2015-01-07,4.8325,4.8750,4.7700,4.7850,4.595531,32180800,-0.002606,neutral
2015-01-08,4.8400,4.9950,4.8375,4.9650,4.768403,28378000,0.037618,up
2015-01-09,4.9825,5.0225,4.9150,4.9850,4.787610,20954000,0.004028,neutral
2015-01-12,4.9975,5.0000,4.8775,4.9225,4.727586,19073200,-0.012537,down
2015-01-13,4.9600,5.0600,4.8800,4.9150,4.720384,23672000,-0.001524,neutral
2015-01-14,4.8625,4.9500,4.8500,4.9350,4.739589,15526000,0.004069,neutral


In [237]:
up_to_up = len(train_data[(train_data["state"] == "up") & (train_data["state"].shift(-1) == "up")]) / len(train_data.query('state == "up"'))
up_to_down = len(train_data[(train_data["state"] == "down") & (train_data["state"].shift(-1) == "up")]) / len(train_data.query('state == "up"'))
up_to_neutral = len(train_data[(train_data["state"] == "neutral") & (train_data["state"].shift(-1) == "up")]) / len(train_data.query('state == "up"'))

down_to_down = len(train_data[(train_data["state"] == "down") & (train_data["state"].shift(-1) == "down")]) / len(train_data.query('state == "down"'))
down_to_up = len(train_data[(train_data["state"] == "up") & (train_data["state"].shift(-1) == "down")]) / len(train_data.query('state == "down"'))
down_to_neutral = len(train_data[(train_data["state"] == "neutral") & (train_data["state"].shift(-1) == "down")]) / len(train_data.query('state == "down"'))

neutral_to_neutral = len(train_data[(train_data["state"] == "neutral") & (train_data["state"].shift(-1) == "neutral")]) / len(train_data.query('state == "neutral"'))
neutral_to_up = len(train_data[(train_data["state"] == "up") & (train_data["state"].shift(-1) == "neutral")]) / len(train_data.query('state == "neutral"'))
neutral_to_down = len(train_data[(train_data["state"] == "down") & (train_data["state"].shift(-1) == "neutral")]) / len(train_data.query('state == "neutral"'))

In [238]:
transition_matrix = pd.DataFrame({
    "up": [up_to_up, up_to_neutral, up_to_down],
    "neutral": [neutral_to_up, neutral_to_neutral, neutral_to_down],
    "down": [down_to_up, down_to_neutral, down_to_down]
}, index=["up", "neutral", "down"])
transition_matrix.head()
print(transition_matrix)

               up   neutral      down
up       0.383895  0.498361  0.423445
neutral  0.254682  0.222951  0.241627
down     0.361423  0.275410  0.334928


In [239]:
train_data.head()

,Open,High,Low,Close,Adj Close,Volume,daily_return,state
Date,,,,,,,,
2015-01-02,5.0325,5.0700,4.9525,5.0325,4.833229,11368000,NaN,neutral
2015-01-05,5.0325,5.0475,4.9250,4.9475,4.751596,19795200,-0.016890,down
2015-01-06,4.9550,4.9600,4.7925,4.7975,4.607536,19776400,-0.030318,down
2015-01-07,4.8325,4.8750,4.7700,4.7850,4.595531,32180800,-0.002606,neutral
2015-01-08,4.8400,4.9950,4.8375,4.9650,4.768403,28378000,0.037618,up


In [240]:
from backtesting import Backtest, Strategy

In [241]:
from enum import Enum

class Trend(Enum):
    DOWN = "Down"
    UP = "Up"
    NEUTRAL = "Neutral"

In [242]:
max_up_values = max(up_to_up, down_to_up, neutral_to_up)
if max_up_values == up_to_up:
    trend_up = Trend.UP
elif max_up_values == down_to_up:
    trend_up = Trend.DOWN
else:
    trend_up = Trend.NEUTRAL

# Determine trend.down
max_down_values = max(up_to_down, down_to_down, neutral_to_down)
if max_down_values == up_to_down:
    trend_down = Trend.UP
elif max_down_values == down_to_down:
    trend_down = Trend.DOWN
else:
    trend_down = Trend.NEUTRAL

print("Based on our Markov Chain from this Stock's past history:")
print(f"We will buy in the {trend_up.name} state. It is the state with the highest chance of going UP next.")
print(f"We will sell in the {trend_down.name} state. It is the state with the highest chance of going DOWN next.")

Based on our Markov Chain from this Stock's past history:
We will buy in the NEUTRAL state. It is the state with the highest chance of going UP next.
We will sell in the UP state. It is the state with the highest chance of going DOWN next.


In [243]:
class MarkovStrategy(Strategy):
    def init(self):
        """init"""

    def get_state(self):
        close_yest = self.data.Close[-2]
        close_today = self.data.Close[-1]

        pct = (close_today - close_yest) / close_yest

        if pct > NEUTRAL_BENCHMARK:
            return Trend.UP
        elif pct < -NEUTRAL_BENCHMARK:
            return Trend.DOWN
        else:
            return Trend.NEUTRAL


    def next(self):
        state = self.get_state()

        self.position.close()
        
        if state == trend_down:
            self.sell()
        elif state == trend_up:
            self.buy()


In [244]:
bt = Backtest(test_data, MarkovStrategy, cash=10000, commission=0)
stats = bt.run()
stats

Start                     2019-12-31 00:00:00
End                       2022-10-19 00:00:00
Duration                   1023 days 00:00:00
Exposure Time [%]                   83.026874
Equity Final [$]                 20802.356266
Equity Peak [$]                  21140.805988
Return [%]                         108.023563
Buy & Hold Return [%]              104.861879
Return (Ann.) [%]                   29.833464
Volatility (Ann.) [%]               52.024349
Sharpe Ratio                         0.573452
Sortino Ratio                        1.153172
Calmar Ratio                          0.84381
Max. Drawdown [%]                  -35.355663
Avg. Drawdown [%]                   -7.835783
Max. Drawdown Duration      434 days 00:00:00
Avg. Drawdown Duration       39 days 00:00:00
# Trades                                  409
Win Rate [%]                        50.122249
Best Trade [%]                      11.743521
Worst Trade [%]                    -13.407333
Avg. Trade [%]                    

In [245]:
bt.plot()


/Users/brandonmonge/Documents/214finalproj/env/lib/python3.11/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Users/brandonmonge/Documents/214finalproj/env/lib/python3.11/site-packages/backtesting/_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
/Users/brandonmonge/Documents/214finalproj/env/lib/python3.11/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
/Users/brandonmonge/Documents/214finalproj/env/lib/python3.11/site-packages/backtesting/_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p5748', ...)

In [246]:
initial_shares = 10000 / test_data["Adj Close"][0]
market_value_after_holding = initial_shares * test_data["Adj Close"][-1]
NAIVE_PROFIT = market_value_after_holding - 10000
print(NAIVE_PROFIT)

10551.863676646175


/var/folders/8w/6tcvtrbs7dncpdlt7gvnbl5c0000gn/T/ipykernel_63707/481196.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  initial_shares = 10000 / test_data["Adj Close"][0]
/var/folders/8w/6tcvtrbs7dncpdlt7gvnbl5c0000gn/T/ipykernel_63707/481196.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  market_value_after_holding = initial_shares * test_data["Adj Close"][-1]


In [247]:
eigenvalues, eigenvectors = np.linalg.eig(transition_matrix)
print("Eigenvalues:", eigenvalues)

# find the index of the Eigenvalue closest to 1
index_of_one = np.argmin(np.abs(eigenvalues - 1))

# Find and normalize the corresponding eigenvector (the stationary distribution)
stationary_vector = eigenvectors[:, index_of_one].real
stationary_vector /= np.sum(stationary_vector)

print("Stationary vector: ", stationary_vector)
print(f"Probability of being in 'up' state overtime: {stationary_vector[0]:.8f}")
print(f"Probability of being 'neutral' state overtime:: {stationary_vector[1]:.8f}")
print(f"Probability of being 'down state overtime:: {stationary_vector[2]:.8f}")


Eigenvalues: [ 0.99920382 -0.0620856   0.00465596]
Stationary vector:  [0.42516062 0.24283536 0.33200402]
Probability of being in 'up' state overtime: 0.42516062
Probability of being 'neutral' state overtime:: 0.24283536
Probability of being 'down state overtime:: 0.33200402
